Dans ce fichier, on définit plusieurs fonctions et méthodes relatifs aux traitements d'un noeud de l'arbre du Branch-and-Price.

On commence par définir le problème maitre (formulation présentée dans le main) :

In [42]:
function master(noeud)
    # Initialisation du probleme maitre
    master = Model(GLPK.Optimizer)
    
    # Definition de la variable z
    # L'indice de z correspond a l'indice du cycle
    z = Array{VariableRef,1}(undef,size(noeud,1))
    for c in 1:size(noeud,1)
        z[c] = @variable(master,lower_bound=0,upper_bound=1)
        set_name(z[c],"z_$c")
    end
    
    # Fonction objective
    @objective(master,Max,sum(noeud[c][1]*z[c] for c in 1:size(noeud,1)))
    
    # Contrainte : Chaque paire participe à au plus un cycle
    
    # Si une paire est dans un cyle, alors le cycle contient un arc dont la paire est à l'origine
    paire = @constraint(master,paire[v in 1:nv(kep)], sum(noeud[c][k+1]*z[c] for c in 1:size(noeud,1), k in origine[v])<=1)
    
    # On retourne le modèle avec ses variables et contraintes
    return paire,z,master
end

LoadError: LoadError: UndefVarError: @variable not defined
in expression starting at In[42]:9

On écrit une fonction qui permet de traiter le sous-problème associé au master.

In [ ]:
function subproblem(π,node)
    SP=Model(GLPK.Optimizer)
    set_optimizer_attribute(SP, "OutputFlag", 0)
    
    # Definition x_a = 1 si l'arc a est dans le cycle retenu, 0 sinon
    @variable(SP,x[a in 1:ne(kep)],Bin)
    
    # Definition de y_i = 1 si paire i dans le cycle retenu, 0 sinon
    @variable(SP,y[i in 1:nv(kep)],Bin)
    
    
    for vertice in 1:nv(kep)
        # Contraintes : les arcs doivent former un cycle
        @constraint(SP,sum(x[a] for a in origine[vertice]) == sum(x[a] for a in dest[vertice]) )
        #Contrainte : y_i vaut 1 ssi il y a un arc qui sort de la paire i qui est choisi dans le cycle
        @constraint(SP, y[vertice] == sum(x[a] for a in origine[vertice]))
    end
    
    # Contrainte : le cycle doit être de longueur inférieure à L 
    @constraint(SP, sum(x[a] for a in 1:ne(kep)) <= L) 
    
    # Objectif : minimiser le coût réduit
    @objective(SP,Min,sum( weights[a]*x[a] for a in 1:ne(kep) ) - sum(π[i]*y[i] for i in 1:nv(kep) ))
    
    # Contraintes permettant de refuser ou d'imposer certains arcs dans la solution
    for arc in tree[node].setzero
        @constraint(SP,x[arc]==0)
    end

    for arcs in tree[node].setone
        @constraint(SP,x[arc]==1)
    end

    optimize!(SP)

    # If the subproblem is solved to optimality then return its objective value
    # and the optimal cycle solution
    # Else the pricing problem is infeasible return Inf, []
    if JuMP.termination_status(SP)==MOI.OPTIMAL
        return JuMP.objective_value(SP),JuMP.value.(y)
    else
        return Inf,[]
    end
end